In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
from io import BytesIO
import requests
from tqdm import tqdm

# Function to fetch image
def fetch_image(image_url):
    try:
        response = requests.get(image_url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        image = image.resize((224, 224))  # Resize image to 224x224
        return image
    except Exception as e:
        print(f"Error fetching {image_url}: {e}")
        return None

# Custom dataset for feature extraction
class CustomDataset(Dataset):
    def __init__(self, df, feature_extractor, transform=None):
        self.df = df
        self.feature_extractor = feature_extractor
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_url = self.df.iloc[idx]['image_link']
        image = fetch_image(image_url)
        if image is None:
            image = Image.new('RGB', (224, 224))  # Create a blank image if fetch fails

        if self.transform:
            image = self.transform(image)
        pixel_values = self.feature_extractor(images=image, return_tensors="pt").pixel_values
        return pixel_values.squeeze(0)

# Define the ViT feature extractor model
class ViTFeatureExtractorModel(nn.Module):
    def __init__(self):
        super(ViTFeatureExtractorModel, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.hidden_size = self.vit.config.hidden_size

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.last_hidden_state[:, 0, :]  # CLS token for each image

# Function to extract features from ViT model
def extract_features(model, dataloader):
    model.eval()
    features = []

    with torch.no_grad():
        for pixel_values in tqdm(dataloader, desc="Extracting Features"):
            pixel_values = pixel_values.to(device)
            feature_rep = model(pixel_values)
            features.append(feature_rep.cpu().numpy())

    return np.concatenate(features)

# Load the dataset and preprocess
test_df = pd.read_csv('/content/test.csv')

# Load the ViT feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Prepare the test dataset and dataloader
test_dataset = CustomDataset(test_df, feature_extractor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model = ViTFeatureExtractorModel().to(device)

# Extract features from the ViT model and save them
test_features = extract_features(vit_model, test_loader)
np.save('test_features.npy', test_features)
print("Test features saved to 'test_features.npy'")


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Extracting Features:   0%|          | 0/4100 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Extracting Features: 100%|██████████| 4100/4100 [1:26:05<00:00,  1.59s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Extracting Features: 100%|██████████| 4100/4100 [1:26:05<00:00,  1.26s/it]


Test features saved to 'test_features.npy'


In [10]:
# Define the CNN model for classification
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import ViTFeatureExtractor, ViTModel
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import requests

class CNNClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        # self.fc3 = nn.Linear(256,256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        # x = self.relu(self.fc3(x))

        x = self.fc3(x)
        return x

# Define the CNN model for regression
class CNNRegressor(nn.Module):
    def __init__(self, input_size):
        super(CNNRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 512)
        # self.fc4 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(512, 1)  # Single output for regression
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        # x = self.relu(self.fc4(x))
        x = self.fc4(x)
        return x


In [ ]:
import numpy as np

# Load the features from the .npy file
features_path = '/content/test_features.npy'  # Replace with your actual path
test_features = np.load(features_path)

# Print the features
print("Test Features:")
print(test_features)

# Print the mean of the features
print("\nMean of Test Features:")
print(np.mean(test_features, axis=0))

print(f"shape = {test_features.shape}")


In [36]:
test_features.dtype

dtype('float32')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv('/content/test.csv')
df.shape

(131187, 4)

In [5]:
feature_df = pd.DataFrame(test_features, columns=[f'feature_{i}' for i in range(test_features.shape[1])])
feature_df.shape

(131187, 768)

In [7]:
final_test_df = pd.concat([feature_df, df], axis=1)
final_test_df.shape

(131187, 772)

In [8]:
final_test_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,index,image_link,group_id,entity_name
0,0.009710,-0.396366,-0.328402,-0.125120,0.007330,0.122043,-0.255793,-0.255737,-0.147798,-0.172934,...,-0.136953,-0.243367,-0.301979,0.105132,0.283707,0.139798,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,0.328610,-0.066665,-0.068489,0.199676,-0.088034,-0.088361,0.068241,0.160914,0.160782,-0.260482,...,-0.174746,0.420909,-0.337411,-0.012518,0.154567,0.231353,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,0.328610,-0.066665,-0.068489,0.199676,-0.088034,-0.088361,0.068241,0.160914,0.160782,-0.260482,...,-0.174746,0.420909,-0.337411,-0.012518,0.154567,0.231353,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,0.328610,-0.066665,-0.068489,0.199676,-0.088034,-0.088361,0.068241,0.160914,0.160782,-0.260482,...,-0.174746,0.420909,-0.337411,-0.012518,0.154567,0.231353,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,0.286884,-0.079419,-0.098814,0.157807,-0.071687,-0.029928,0.092473,0.131691,0.161884,-0.283289,...,-0.191619,0.368619,-0.349722,0.026817,0.025484,0.180161,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [11]:
model_path_dict = {
    'depth': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_depth.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_depth.pth'],
    'height':['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_height.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_height.pth'],
    'item_volume':['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_volume.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_volume.pth'],
    'item_weight': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_item_weight.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_item_weight.pth'],
    'maximum_weight_recommendation': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_max_weight_recommendation.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_max_weight_recommendation.pth'],
    'voltage': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_voltage.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_voltage.pth'],
    'wattage': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_wattage.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_wattage.pth'],
    'width':['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_width.pth','/content/drive/MyDrive/AmazonMLHack/best_regressor_model_width.pth']
}


In [25]:
mod = pd.read_csv('/content/modified_file_2.csv')
mod.head()
ptu = pd.read_csv('/content/processed_train_unit.csv')
ptu.head()

,Unnamed: 0,image_link,group_id,entity_name,entity_value_nos,entity_value_units
0,0,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.535908,gram
1,1,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.535908,gram
2,2,https://m.media-amazon.com/images/I/81N73b5khV...,149159,item_weight,10.308986,kilogram
3,3,https://m.media-amazon.com/images/I/61oMj2iXOu...,308856,item_weight,9.433564,kilogram
4,4,https://m.media-amazon.com/images/I/91LPf6OjV9...,281678,item_weight,4.615850,ounce


In [118]:
print(model_path_dict)

{'depth': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_depth.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_depth.pth'], 'height': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_height.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_height.pth'], 'item_volume': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_volume.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_volume.pth'], 'item_weight': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_item_weight.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_item_weight.pth'], 'maximum_weight_recommendation': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_max_weight_recommendation.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_max_weight_recommendation.pth'], 'voltage': ['/content/drive/MyDrive/AmazonMLHack/best_classifier_model_voltage.pth', '/content/drive/MyDrive/AmazonMLHack/best_regressor_model_vol

In [117]:
model_dict = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for key, value in model_path_dict.items():
    print(f" key = {key}")
    # udf = ptu[ptu['entity_name'] == key]
    value = model_path_dict[key]
    clf_dict = torch.load(value[0],  map_location=device)
    reg_dict = torch.load(value[1],  map_location=device)

    num_classes = clf_dict['fc3.weight'].size(0)


    clf = CNNClassifier(input_size=test_features.shape[1], num_classes=num_classes).to(device)
    reg = CNNRegressor(input_size=test_features.shape[1]).to(device)

    clf.load_state_dict(clf_dict)
    reg.load_state_dict(reg_dict)
    model_dict[key] = [clf,reg]


 key = depth
 key = height
 key = item_volume
 key = item_weight


<ipython-input-117-93051b2600a6>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf_dict = torch.load(value[0],  map_location=device)
<ipython-input-117-93051b2600a6>:8: F

 key = maximum_weight_recommendation
 key = voltage
 key = wattage
 key = width


In [129]:
encod_dict = {}

stunames = ['item_volume','voltage','maximum_weight_recommendation','wattage']
modnames = ['item_weight','width','depth','height']

entity_names = df['entity_name'].unique()

scMod = StandardScaler()
scPtu = StandardScaler()

leMod = LabelEncoder()
lePtu = LabelEncoder()

scMod.fit(mod[['entity_value_nos']])
scPtu.fit(ptu[['entity_value_nos']])
leMod.fit(mod['entity_value_units'])
lePtu.fit(ptu['entity_value_units'])

# for emtity_name in entity_names:
#     if emtity_name in modnames:
#         encod_dict[emtity_name] = [scMod,leMod]
#     else:
#         encod_dict[emtity_name] = [scPtu,lePtu]

# for int

for entity_name in entity_names:
  sc = StandardScaler()
  le = LabelEncoder()
  if entity_name in modnames:
    dfe = mod[mod['entity_name'] == entity_name]
    ue = dfe['entity_value_units'].unique()
    sc.fit(dfe[['entity_value_nos']])
    le.fit(ue)
    encod_dict[entity_name] = [sc,le]

  else:
    dfe = ptu[ptu['entity_name'] == entity_name]
    ue = dfe['entity_value_units'].unique()
    sc.fit(dfe[['entity_value_nos']])
    le.fit(ue)
    encod_dict[entity_name] = [sc,le]




In [130]:
# Define the conversion factor dictionary (as shown above)
conversion_factors = {
    'gram': 1, 'kilogram': 1000, 'milligram': 0.001, 'microgram': 1e-6, 'ton': 1e6, 'ounce': 28.3495, 'pound': 453.592,
    'millilitre': 1, 'litre': 1000, 'cup': 240, 'fluid ounce': 29.5735, 'quart': 946.353, 'gallon': 3785.41,
    'cubic foot': 28316.8, 'cubic inch': 16.3871, 'decilitre': 100, 'centilitre': 10, 'pint': 473.176,
    'centimetre': 1, 'metre': 100, 'millimetre': 0.1, 'inch': 2.54, 'foot': 30.48,
    'volt': 1, 'watt': 1, 'kilowatt': 1000
}

# These are the max occurring units for each entity_name
standard_units = {
    'depth': 'centimetre',
    'height': 'centimetre',
    'item_volume': 'millilitre',
    'item_weight': 'gram',
    'maximum_weight_recommendation': 'kilogram',
    'voltage': 'volt',
    'wattage': 'watt',
    'width': 'centimetre'
}

def inverse_transform(predicted_value, predicted_unit, entity_name):
    """
    Convert the standardized predicted_value back to the predicted_unit.

    predicted_value: The value predicted by the regression head in the standard unit.
    predicted_unit: The unit predicted by the classification head.
    entity_name: The type of entity (e.g., 'item_weight', 'height', etc.).
    """
    standard_unit = standard_units[entity_name]  # Lookup the standard unit for the entity (e.g., 'kilogram')

    if predicted_unit in conversion_factors and standard_unit in conversion_factors:
        # Calculate the inverse conversion factor
        inverse_conversion_factor = conversion_factors[standard_unit] / conversion_factors[predicted_unit]
        # Apply inverse transformation to convert from standard unit to predicted unit
        return predicted_value * inverse_conversion_factor
    else:
        return predicted_value  # If no valid conversion, return the original value

# Assuming df now contains the 'predicted_value' (standardized) and 'predicted_unit' columns
# df['final_value'] = df.apply(lambda row: inverse_transform(row['predicted_value'],
#                                                           row['predicted_unit'],
#                                                           row['entity_name']), axis=1)

In [121]:
for i in range(27):
  print(lePtu.inverse_transform([i]))

['centilitre']
['centimetre']
['cubic foot']
['cubic inch']
['cup']
['decilitre']
['fluid ounce']
['foot']
['gallon']
['gigabyte']
['gram']
['inch']
['kilogram']
['kilowatt']
['litre']
['metre']
['microgram']
['milligram']
['millilitre']
['millimetre']
['ounce']
['pint']
['pound']
['quart']
['ton']
['volt']
['watt']


In [131]:

def predict_single_row(row, device):
    features = row.drop(df.columns).values  # Assuming entity_name is not a feature
    # print(features.shape)
    entity = row['entity_name']
    features = features.astype(np.float32)
    print(f"entity = {entity}")

    scaler = encod_dict[entity][0]
    label_encoder = encod_dict[entity][1]
    classification_model = model_dict[entity][0]
    regression_model = model_dict[entity][1]

    regression_model.eval()
    with torch.no_grad():
        features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
        predicted_value = regression_model(features_tensor).cpu().numpy().reshape(-1, 1)

    # Inverse transform to get the original scale of entity values
    original_value = scaler.inverse_transform(predicted_value)
    original_value = np.expm1(original_value)
    # print(f"Predicted numerical value (Original Scale): {original_value[0][0]}")


    # Predict the entity unit (classification)
    classification_model.eval()
    with torch.no_grad():
        predicted_unit = classification_model(features_tensor)
        print(f" pred unit shape{predicted_unit.shape}")
        print(f" pred unit {predicted_unit}")
        pc = predicted_unit.cpu().numpy()
        print(f" pred unit cpu {pc}")
        predicted_class = np.argmax(pc)
        # predicted_class =  torch.argmax(predicted_unit, dim=0).cpu().numpy()
        predicted_class = [predicted_class]

        print(f" pred class {predicted_class}")


    # Convert the predicted class back to the original unit label
    # print("into the labelencoder")
    predicted_unit_label = label_encoder.inverse_transform(predicted_class)
    print(f"Predicted unit (Classification): {predicted_unit_label[0]}")

    return f"{original_value[0][0]} { predicted_unit_label[0]}"



In [124]:
final_test_df = final_test_df.drop(columns=['predicted_value'])

In [132]:
row1 = final_test_df.iloc[0]
r = row1.drop(df.columns).values.astype(np.float32)
print(f"rdtype = {r.dtype} \n {r}")

rdtype = float32 
 [ 9.71028209e-03 -3.96365970e-01 -3.28401655e-01 -1.25120461e-01
  7.32982997e-03  1.22042663e-01 -2.55793184e-01 -2.55737066e-01
 -1.47797689e-01 -1.72933817e-01  4.99425195e-02 -1.75874427e-01
 -6.36492148e-02 -2.90459275e-01 -2.50883669e-01  1.58569500e-01
 -1.55713826e-01  1.18061878e-01  4.85568047e-02 -3.55040766e-02
 -9.24069956e-02  9.23197940e-02  1.75654013e-02  1.79923311e-01
  2.04919666e-01  2.31671810e-01 -1.26499206e-01  4.98338640e-02
  2.66743094e-01 -2.31228575e-01  1.29817188e-01 -1.65556565e-01
  2.54502386e-01 -1.86750554e-02 -8.14905688e-02 -2.57195711e-01
  1.83040038e-01  2.16009080e-01  1.15582824e-01 -1.90222114e-01
 -1.17685601e-01  1.08934723e-01  1.57647476e-01 -1.41248420e-01
  2.38578990e-02 -4.31955159e-02 -2.14778230e-01  2.59010166e-01
 -2.46285088e-02  1.37933502e-02  4.32877600e-01 -9.45964232e-02
  3.32001597e-02 -2.25181773e-01  2.84648925e-01  1.96455903e-02
  2.00572103e-01  3.99177849e-01  1.44715905e-01  3.37428808e-01
  5.67

In [133]:
row1 = final_test_df.iloc[0]

predict_single_row(row1,device)

entity = height
 pred unit shapetorch.Size([5])
 pred unit tensor([ -1.0601, -20.6800,  -1.1651,  -9.6280,  -1.7792])
 pred unit cpu [ -1.0600661 -20.680029   -1.1651376  -9.628036   -1.7792188]
 pred class [0]
Predicted unit (Classification): centimetre


'14.50595760345459 centimetre'

In [134]:
final_test_df['predicted_value'] = final_test_df.apply(lambda row: predict_single_row(row, device), axis=1)

Streaming output truncated to the last 5000 lines.
        -31.9840, -32.4389, -13.1085,   6.7403,  -8.8303])
 pred unit cpu [-29.121155  -36.03588    -2.4884684 -37.7004      1.6043624 -19.276684
 -10.356385  -31.984015  -32.43888   -13.10852     6.7402854  -8.830257 ]
 pred class [10]
Predicted unit (Classification): pound
entity = item_weight
 pred unit shapetorch.Size([12])
 pred unit tensor([-50.5611, -60.7610,   4.0967, -79.7105,   6.5824, -14.5353,   2.4319,
        -26.5824, -70.7460, -17.6491, -12.4174, -27.7135])
 pred unit cpu [-50.5611    -60.761       4.096745  -79.71055     6.5823746 -14.535343
   2.431898  -26.582355  -70.74597   -17.649136  -12.417428  -27.713507 ]
 pred class [4]
Predicted unit (Classification): kilogram
entity = item_weight
 pred unit shapetorch.Size([12])
 pred unit tensor([-50.5283, -86.7318, -16.1393, -96.5093,  11.7507, -45.0428,  -7.3514,
        -68.3416, -71.2543, -27.8697,   8.9255, -17.6738])
 pred unit cpu [-50.528313  -86.73184   -16.139254

In [135]:
final_test_df.sample(10)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_763,feature_764,feature_765,feature_766,feature_767,index,image_link,group_id,entity_name,predicted_value
6406,-0.042403,-0.195242,-0.155925,0.305253,-0.184646,0.013577,-0.246322,-0.340812,0.130510,-0.253820,...,-0.095291,-0.325389,0.029950,0.034683,0.228131,6416,https://m.media-amazon.com/images/I/41GT8stJ76...,599772,width,18.91615867614746 inch
107102,-0.388005,0.032997,-0.285607,0.254519,-0.093162,-0.290170,0.424500,0.198933,0.170840,0.334745,...,-0.089139,0.033392,-0.222386,0.005148,-0.131728,107178,https://m.media-amazon.com/images/I/61FaNzI5yp...,844474,height,15.108743667602539 inch
71853,-0.237777,-0.136423,-0.036463,0.297155,0.091717,-0.118852,-0.054335,-0.211196,-0.288270,-0.048781,...,0.171340,0.012219,0.212706,0.079397,0.185255,71918,https://m.media-amazon.com/images/I/51tzpBlgLq...,731252,width,4.981016635894775 centimetre
69914,0.377403,-0.340544,-0.239780,-0.169090,0.044774,-0.426599,0.455855,-0.284882,-0.087709,-0.096937,...,0.646439,0.091092,-0.074625,0.213627,0.065789,69978,https://m.media-amazon.com/images/I/51rmP0QF67...,648011,height,11.346435546875 inch
95887,-0.003828,0.030969,-0.013409,-0.181477,0.225850,-0.191322,-0.017911,-0.060949,-0.421149,-0.150482,...,-0.191324,0.346122,-0.003357,-0.249736,0.235441,95961,https://m.media-amazon.com/images/I/619+TesFMb...,653767,width,18.67144203186035 centimetre
98756,0.108916,-0.025871,-0.093992,0.128031,-0.131044,-0.282826,0.008203,-0.053238,-0.028603,-0.140286,...,-0.108512,0.180729,0.138748,0.206052,0.231981,98830,https://m.media-amazon.com/images/I/61Aih433W0...,965725,width,6.408518314361572 centimetre
105897,-0.044177,-0.190189,0.082128,0.174293,-0.515213,0.273294,-0.081305,0.141464,-0.005503,-0.024997,...,0.196522,-0.026871,-0.143140,0.010491,0.002369,105973,https://m.media-amazon.com/images/I/61EumEEvEP...,784538,depth,20.498828887939453 centimetre
63224,-0.159501,-0.025249,-0.259789,-0.022412,-0.082300,-0.310965,0.070751,-0.346314,0.020267,-0.414554,...,0.153314,-0.152398,-0.070362,-0.098201,0.236285,63277,https://m.media-amazon.com/images/I/51kY7KWLVa...,470067,height,10.984289169311523 inch
124198,-0.156210,-0.042897,-0.115969,0.015180,0.190381,-0.503362,0.102242,0.237487,-0.109526,0.080258,...,0.213120,-0.321395,0.119493,0.072530,0.250543,124290,https://m.media-amazon.com/images/I/71snJFIAaN...,892291,wattage,48.787132263183594 watt
62198,-0.036261,-0.385897,0.227159,-0.238980,-0.100129,0.345351,-0.092396,-0.137875,-0.064774,-0.252342,...,0.074795,-0.304519,0.235679,-0.025637,0.065490,62250,https://m.media-amazon.com/images/I/51jNUMciT3...,237000,height,23.849559783935547 centimetre


In [136]:
final_test_df.shape

(131187, 773)

In [137]:
final_test_df.to_csv('predicted_values_final.csv', index=False)

In [148]:
submission = final_test_df['predicted_value']
# final_test_df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
submission = submission.rename('prediction')
submission.head()
submission.to_csv('submission.csv', index=True)